In [2]:
# Load silter sales table to df

df = spark.read.table("Sales.sales_silver")

display(df)


StatementMeta(, c61c4e71-26fa-497e-9953-2e77bca371de, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5879a894-e917-44d2-8e6f-5903201b8f8d)

In [3]:
from pyspark.sql.types import *
from delta.tables import *

# Define schema for dim date for gold layer
DeltaTable.createIfNotExists(spark)\
          .tableName("sales.dimdate_gold")\
          .addColumn("OrderDate", DateType())\
          .addColumn("Day", IntegerType())\
          .addColumn("Month", IntegerType())\
          .addColumn("Year", IntegerType())\
          .addColumn("mmmyyyy", StringType())\
          .addColumn("yyyymm", StringType())\
          .execute()
          

StatementMeta(, c61c4e71-26fa-497e-9953-2e77bca371de, 5, Finished, Available, Finished)

In [11]:
from pyspark.sql.functions import *

# Create df for dimdate_gold

dfdimDate_gold = df.dropDuplicates(["OrderDate"])\
                   .select(col("OrderDate"),
                           dayofmonth(col("OrderDate")).alias("Day"),
                           month(col("OrderDate")).alias("Month"),
                           year(col("OrderDate")).alias("Year"),
                           date_format(col("OrderDate"), "MMM-yyyy").alias("mmmyyyy"),
                           date_format(col("OrderDate"), "yyyyMM").alias("yyyymm"))\
                   .orderBy(col("OrderDate"))

display(dfdimDate_gold)

StatementMeta(, c61c4e71-26fa-497e-9953-2e77bca371de, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 307cac3e-2713-4bf2-8749-bf16a7ed01aa)

In [12]:
dfdimDate_gold.write.format("delta").mode("overwrite").option("mergeSchema", "true").saveAsTable("sales.dimdate_gold")

StatementMeta(, c61c4e71-26fa-497e-9953-2e77bca371de, 14, Finished, Available, Finished)

In [13]:
from delta.tables import *
    
deltaTable = DeltaTable.forPath(spark, 'Tables/dimdate_gold')
    
dfUpdates = dfdimDate_gold
    
deltaTable.alias('gold') \
  .merge(
    dfUpdates.alias('updates'),
    'gold.OrderDate = updates.OrderDate'
  ) \
   .whenMatchedUpdate(set =
    {
          
    }
  ) \
 .whenNotMatchedInsert(values =
    {
      "OrderDate": "updates.OrderDate",
      "Day": "updates.Day",
      "Month": "updates.Month",
      "Year": "updates.Year",
      "mmmyyyy": "updates.mmmyyyy",
      "yyyymm": "updates.yyyymm"
    }
  ) \
  .execute()

StatementMeta(, c61c4e71-26fa-497e-9953-2e77bca371de, 15, Finished, Available, Finished)

In [14]:
DeltaTable.createIfNotExists(spark)\
          .tableName("sales.dimcustomer_gold")\
          .addColumn("CustomerName", StringType())\
          .addColumn("Email", StringType())\
          .addColumn("First", StringType())\
          .addColumn("Last", StringType())\
          .addColumn("CustomerID", LongType())\
          .execute()

StatementMeta(, c61c4e71-26fa-497e-9953-2e77bca371de, 16, Finished, Available, Finished)

In [15]:
dfdimCustomer_silver = df.dropDuplicates(["CustomerName", "Email"])\
                         .select(col("CustomerName"),
                                 col("Email"),
                                 split(col("CustomerName")," ").getItem(0).alias("First"),
                                 split(col("CustomerName")," ").getItem(1).alias("Last")
                         )

display(dfdimCustomer_silver)                         

StatementMeta(, c61c4e71-26fa-497e-9953-2e77bca371de, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7b234a58-f222-493f-b2d6-348ab73cb9c6)

In [17]:
from pyspark.sql.functions import monotonically_increasing_id, col, when, coalesce, max, lit
    
dfdimCustomer_temp = spark.read.table("Sales.dimCustomer_gold")
    
MAXCustomerID = dfdimCustomer_temp.select(coalesce(max(col("CustomerID")),lit(0)).alias("MAXCustomerID")).first()[0]
    
dfdimCustomer_gold = dfdimCustomer_silver.join(dfdimCustomer_temp,(dfdimCustomer_silver.CustomerName == dfdimCustomer_temp.CustomerName) & (dfdimCustomer_silver.Email == dfdimCustomer_temp.Email), "left_anti")
    
dfdimCustomer_gold = dfdimCustomer_gold.withColumn("CustomerID",monotonically_increasing_id() + MAXCustomerID + 1)

# Display the first 10 rows of the dataframe to preview your data

display(dfdimCustomer_gold)

StatementMeta(, c61c4e71-26fa-497e-9953-2e77bca371de, 19, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 56cbc07e-8570-4634-810c-ab9995483d5b)

In [18]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, 'Tables/dimcustomer_gold')
    
dfUpdates = dfdimCustomer_gold
    
deltaTable.alias('gold') \
  .merge(
    dfUpdates.alias('updates'),
    'gold.CustomerName = updates.CustomerName AND gold.Email = updates.Email'
  ) \
   .whenMatchedUpdate(set =
    {
          
    }
  ) \
 .whenNotMatchedInsert(values =
    {
      "CustomerName": "updates.CustomerName",
      "Email": "updates.Email",
      "First": "updates.First",
      "Last": "updates.Last",
      "CustomerID": "updates.CustomerID"
    }
  ) \
  .execute()

StatementMeta(, c61c4e71-26fa-497e-9953-2e77bca371de, 20, Finished, Available, Finished)

In [19]:
DeltaTable.createIfNotExists(spark)\
          .tableName("sales.dimproduct_gold")\
          .addColumn("ItemName", StringType())\
          .addColumn("ItemID", LongType())\
          .addColumn("ItemInfo", StringType())\
          .execute()

StatementMeta(, c61c4e71-26fa-497e-9953-2e77bca371de, 21, Finished, Available, Finished)

In [21]:
from pyspark.sql.functions import col, split, lit, when
    
# Create product_silver dataframe
    
dfdimProduct_silver = df.dropDuplicates(["Item"])\
                        .select(col("Item")) \
                        .withColumn("ItemName",split(col("Item"), ", ").getItem(0)) \
                        .withColumn("ItemInfo",when((split(col("Item"), ", ").getItem(1).isNull() | (split(col("Item"), ", ").getItem(1)=="")),lit("")).otherwise(split(col("Item"), ", ").getItem(1))) 
    
# Display the first 10 rows of the dataframe to preview your data

display(dfdimProduct_silver.head(10))

StatementMeta(, c61c4e71-26fa-497e-9953-2e77bca371de, 23, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 95dee031-8eba-49cd-8ef5-bf5210ce087b)

In [22]:
from pyspark.sql.functions import monotonically_increasing_id, col, lit, max, coalesce
    
#dfdimProduct_temp = dfdimProduct_silver
dfdimProduct_temp = spark.read.table("Sales.dimProduct_gold")
    
MAXProductID = dfdimProduct_temp.select(coalesce(max(col("ItemID")),lit(0)).alias("MAXItemID")).first()[0]
    
dfdimProduct_gold = dfdimProduct_silver.join(dfdimProduct_temp,(dfdimProduct_silver.ItemName == dfdimProduct_temp.ItemName) & (dfdimProduct_silver.ItemInfo == dfdimProduct_temp.ItemInfo), "left_anti")
    
dfdimProduct_gold = dfdimProduct_gold.withColumn("ItemID",monotonically_increasing_id() + MAXProductID + 1)
    
# Display the first 10 rows of the dataframe to preview your data

display(dfdimProduct_gold.head(10))

StatementMeta(, c61c4e71-26fa-497e-9953-2e77bca371de, 24, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2dd43e02-8619-4e19-a14f-7e7f3ca127fb)

In [24]:
deltaTable = DeltaTable.forPath(spark, "Tables/dimproduct_gold")

dfUpdates = dfdimProduct_gold

deltaTable.alias("gold")\
          .merge(dfUpdates.alias("updates"),
                "gold.ItemName = updates.ItemName AND gold.ItemInfo = updates.ItemInfo")\
          .whenMatchedUpdate(set={})\
          .whenNotMatchedInsert(values = 
            {
                "ItemName":"updates.ItemName",
                "ItemInfo":"updates.ItemInfo",
                "ItemID":"updates.ItemID"
            })\
          .execute()

StatementMeta(, c61c4e71-26fa-497e-9953-2e77bca371de, 26, Finished, Available, Finished)

In [25]:
from pyspark.sql.types import *
from delta.tables import *
    
DeltaTable.createIfNotExists(spark) \
    .tableName("sales.factsales_gold") \
    .addColumn("CustomerID", LongType()) \
    .addColumn("ItemID", LongType()) \
    .addColumn("OrderDate", DateType()) \
    .addColumn("Quantity", IntegerType()) \
    .addColumn("UnitPrice", FloatType()) \
    .addColumn("Tax", FloatType()) \
    .execute()

StatementMeta(, c61c4e71-26fa-497e-9953-2e77bca371de, 27, Finished, Available, Finished)

In [26]:
dfdimCustomer_temp = spark.read.table("Sales.dimcustomer_gold")
dfdimProduct_temp = spark.read.table("Sales.dimproduct_gold")

df = df.withColumn("ItemName",split(col("Item"), ", ").getItem(0)) \
       .withColumn("ItemInfo",when((split(col("Item"), ", ").getItem(1).isNull() | (split(col("Item"), ", ").getItem(1)=="")),lit("")).otherwise(split(col("Item"), ", ").getItem(1)))

display(df)
    

StatementMeta(, c61c4e71-26fa-497e-9953-2e77bca371de, 28, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4cb0c3c2-025a-4876-ad68-2d66dad311a2)

In [29]:
dffactSales_gold = df.alias("df1").join(dfdimCustomer_temp.alias("df2"),
                                        (col("df1.CustomerName") == col("df2.CustomerName")) &
                                        (col("df1.Email") == col("df2.Email")), "left")\
                                  .join(dfdimProduct_temp.alias("df3"),
                                        (col("df1.ItemName") == col("df3.ItemName")) &
                                        (col("df1.ItemInfo") == col("df3.ItemInfo")), "left")\
                                  .select(col("df2.CustomerID"),
                                          col("df3.ItemID"),
                                          col("df1.OrderDate"),
                                          col("df1.Quantity"),
                                          col("df1.UnitPrice"),
                                          col("df1.Tax"))\
                                  .orderBy(col("df1.OrderDate"),
                                           col("df2.CustomerID"),
                                           col("df3.ItemID"))

display(dffactSales_gold)                                           

StatementMeta(, c61c4e71-26fa-497e-9953-2e77bca371de, 31, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 489efdf8-a7fe-4e1b-85ec-4e4e45ff5c65)

In [31]:
deltaTable = DeltaTable.forPath(spark, "Tables/factsales_gold")

dfUpdates = dffactSales_gold

deltaTable.alias("gold")\
          .merge(dfUpdates.alias("updates"),
            "gold.OrderDate = updates.OrderDate AND \
             gold.CustomerID = updates.CustomerID AND \
             gold.ItemID = updates.ItemID")\
          .whenMatchedUpdate(set={})\
          .whenNotMatchedInsert(values=
            {
                "CustomerID":"updates.CustomerID",
                "ItemID":"updates.ItemID",
                "OrderDate":"updates.OrderDate",
                "Quantity":"updates.Quantity",
                "UnitPrice":"updates.UnitPrice",
                "Tax":"updates.Tax"
            })\
          .execute()

StatementMeta(, c61c4e71-26fa-497e-9953-2e77bca371de, 33, Finished, Available, Finished)